In [ ]:
# This code is inspired by the blog Building AI-powered apps on Google Cloud databases using pgvector, LLMs and LangChain
# https://cloud.google.com/blog/products/databases/using-pgvector-llms-and-langchain-with-google-cloud-databases/

# Set up

In [ ]:
# Install dependencies.
!pip install -qU asyncio==3.4.3 asyncpg==0.27.0 cloud-sql-python-connector["asyncpg"]==1.2.3
!pip install -qU numpy==1.22.4 pandas==1.5.3
!pip install -qU pgvector==0.1.8
!pip install -qU langchain==0.3.0 transformers==4.48.0
!pip install -qU google-cloud-aiplatform==1.76.0
!pip install -qU langchain-google-vertexai==2.0.11
!pip install -qU shapely==1.8.5.post1 pygeos==0.12.0 geopandas==0.10.2
!pip install -qU langchain-huggingface==0.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 15.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Cannot install numpy==1.22.4 and pandas==1.5.3 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.1

In [ ]:
# Automatically restart kernel after installs
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
# @ ###markdown Google Cloud SQL instance parameters

project_id = "chatbot-with-rag-447503"  # @param {type:"string"}
database_password = "19991028"  # @param {type:"string"}
region = "us-central1"  # @param {type:"string"}
instance_name = "pg15-embeddings-pgvector-demo"  # @param {type:"string"}
database_name = "clothing"  # @param {type:"string"}
database_user = "clothing-admin"  # @param {type:"string"}


# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"
assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"

In [ ]:
#@markdown ###Authenticate Google Cloud Account and enable APIs.
# Authenticate gcloud.
from google.colab import auth
auth.authenticate_user()

# Configure gcloud.
!gcloud config set project {project_id}

# Grant Cloud SQL Client role to authenticated user
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

# Enable Cloud SQL Admin API
!gcloud services enable sqladmin.googleapis.com
!gcloud services enable aiplatform.googleapis.com

Updated property [core/project].
Updated IAM policy for project [chatbot-with-rag-447503].
bindings:
- members:
  - user:jiexingluo0418@gmail.com
  role: roles/cloudsql.client
- members:
  - user:jiexingluo0418@gmail.com
  role: roles/owner
etag: BwYr8_EhNvY=
version: 1


In [ ]:
#@markdown Create and setup a Cloud SQL PostgreSQL instance, if not done already.
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("POSTGRES"):
  print("Found an existing Postgres Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  !gcloud sql instances create {instance_name} --database-version=POSTGRES_15 \
    --region={region} --cpu=1 --memory=4GB --root-password={database_password}

# Create the database, if it does not exist.
out = !gcloud sql databases list --instance={instance_name} --filter="NAME:{database_name}" --format="value(NAME)"
if ''.join(out) == database_name:
  print("Database %s already exists, skipping creation." % database_name)
else:
  !gcloud sql databases create {database_name} --instance={instance_name}

# Create the database user for accessing the database.
!gcloud sql users create {database_user} \
  --instance={instance_name} \
  --password={database_password}

Found an existing Postgres Cloud SQL Instance!
Database clothing already exists, skipping creation.
Created user [clothing-admin].


In [ ]:
# @markdown Verify connection to the database. Executing this block should print the current PostgreSQL server version.

import asyncio
import asyncpg
from google.cloud.sql.connector import Connector


async def main():
    # get current running event loop to be used with Connector
    loop = asyncio.get_running_loop()
    # initialize Connector object as async context manager
    async with Connector(loop=loop) as connector:
        # create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}"
            # ... additional database driver args
        )

        # query Cloud SQL database
        results = await conn.fetch("SELECT version()")
        print(results[0]["version"])

        # close asyncpg connection
        await conn.close()


# Test connection with `asyncio`
await main()  # type: ignore

/usr/local/lib/python3.11/dist-packages/google/cloud/sql/connector/instance.py:330: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  expiration = x509.not_valid_after


PostgreSQL 15.10 on x86_64-pc-linux-gnu, compiled by Debian clang version 12.0.1, 64-bit


# Prepare data

In [ ]:
# Load dataset from a web URL and store it in a pandas dataframe.
# pandas is kind of like a table
import pandas as pd
import os

import kagglehub

# Download latest version
path = kagglehub.dataset_download("nicapotato/womens-ecommerce-clothing-reviews")

print("Path to dataset files:", path)

df = pd.read_csv(f"{path}/Womens Clothing E-Commerce Reviews.csv")
df = df.loc[:, ["Unnamed: 0", "Clothing ID", "Age", "Title", "Review Text", "Rating"]] # selects column

# Rename the columns
df = df.rename(columns={
    "Unnamed: 0": "index",
    "Clothing ID": "clothing_id",  # Lowercase with underscore
    "Age": "age",
    "Title": "title",
    "Review Text": "review_text",
    "Rating": "rating"
})

# df = df.dropna() # removes rows or columns with missing values
# Convert 'col1' to string
df['clothing_id'] = df['clothing_id'].astype(str)
df = df.fillna('N/A')
df['review'] = df['title'].str.cat(df['review_text'], sep='-')  # Use '-' as a separator

del df['title']
del df['review_text']


df.head(10) # shows the first 10
# print(df.info())

100%|██████████| 2.79M/2.79M [00:00<00:00, 151MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/nicapotato/womens-ecommerce-clothing-reviews/versions/1


,index,clothing_id,age,rating,review
0,0,767,33,4,N/A-Absolutely wonderful - silky and sexy and ...
1,1,1080,34,5,N/A-Love this dress! it's sooo pretty. i hap...
2,2,1077,60,3,Some major design flaws-I had such high hopes ...
3,3,1049,50,5,"My favorite buy!-I love, love, love this jumps..."
4,4,847,47,5,Flattering shirt-This shirt is very flattering...
5,5,1080,49,2,Not for the very petite-I love tracy reese dre...
6,6,858,39,5,Cagrcoal shimmer fun-I aded this in my basket ...
7,7,858,39,4,"Shimmer, surprisingly goes with lots-I ordered..."
8,8,1077,24,5,Flattering-I love this dress. i usually get an...
9,9,1077,34,5,"Such a fun dress!-I'm 5""5' and 125 lbs. i orde..."


In [ ]:
# Save the Pandas dataframe in a PostgreSQL table.

import asyncio
import asyncpg
from google.cloud.sql.connector import Connector


async def main():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await conn.execute("DROP TABLE IF EXISTS clothing CASCADE")
        # Create the `products` table.
        await conn.execute( # SQL here
            """CREATE TABLE clothing(
                                index INTEGER PRIMARY KEY,
                                clothing_id VARCHAR(1024),
                                age INTEGER,
                                rating NUMERIC,
                                review TEXT)"""
        )

        # Copy the dataframe to the `products` table.
        tuples = list(df.itertuples(index=False))
        await conn.copy_records_to_table(
            "clothing", records=tuples, columns=list(df), timeout=10
        )
        await conn.close()


# Run the SQL commands now.
await main()  # type: ignore

/usr/local/lib/python3.11/dist-packages/google/cloud/sql/connector/instance.py:330: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  expiration = x509.not_valid_after


# Create embeddings

In [ ]:
# Split long text descriptions into smaller chunks that can fit into
# the API request size limit, as expected by the LLM providers.

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=[], # was [".", "\n"],
    chunk_size=1000, # 500 would create duplicate items
    chunk_overlap=0,
    length_function=len,
)

chunked = []
for index, row in df.iterrows():
    clothing_id = row["clothing_id"]
    desc = row["review"]
    splits = text_splitter.create_documents([desc])
    for s in splits:
        r = {"clothing_id": clothing_id, "content": s.page_content}
        chunked.append(r)

In [ ]:
# use hugging face and embed chunk locally
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from google.cloud import aiplatform
import time
import torch

aiplatform.init(project=f"{project_id}", location=f"{region}")

# Alternative: use vertaxAI for embedding, extremely slow
# embeddings_service = VertexAIEmbeddings(model="text-embedding-004")

# run embedding model locally
# "sentence-transformers/all-MiniLM-L6-v2" 384 dimensional vector space,
# took 20 mins to run on cpu
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {
    "device": "cuda" if torch.cuda.is_available() else "cpu",
   # "show_progress": True  # Enable progress bar
}
if torch.cuda.is_available():
  print('encoding with cuda')
else:
  print('encoding with cpu')
embeddings_service = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs)

# Helper function to retry failed API requests with exponential backoff.
def retry_with_backoff(func, *args, retry_delay=5, backoff_factor=2, **kwargs):
    max_attempts = 10
    retries = 0
    for i in range(max_attempts):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"error: {e}")
            retries += 1
            wait = retry_delay * (backoff_factor**retries)
            print(f"Retry after waiting for {wait} seconds...")
            time.sleep(wait)


batch_size = 5
for i in range(0, len(chunked), batch_size):
    request = [x["content"] for x in chunked[i : i + batch_size]] # list comprehension
    response = retry_with_backoff(embeddings_service.embed_documents, request)
    print("processing")
    #time.sleep(1)
    # Store the retrieved vector embeddings for each chunk back.
    for x, e in zip(chunked[i : i + batch_size], response): # zip: joins the elements of two lists at the same position
        x["embedding"] = e

# Store the generated embeddings in a pandas dataframe.
product_embeddings = pd.DataFrame(chunked)
product_embeddings = product_embeddings.reset_index() # indexing
product_embeddings.head()

In [ ]:
# Store the generated vector embeddings in a PostgreSQL table.
import asyncio
import asyncpg
from google.cloud.sql.connector import Connector
import numpy as np
from pgvector.asyncpg import register_vector


async def main():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await conn.execute("CREATE EXTENSION IF NOT EXISTS vector")
        await register_vector(conn)

        await conn.execute("DROP TABLE IF EXISTS clothing_embeddings")
        # Create the `product_embeddings` table to store vector embeddings.
        await conn.execute(
            """CREATE TABLE clothing_embeddings(
                                index INTEGER NOT NULL REFERENCES clothing(index),
                                clothing_id VARCHAR(1024),
                                content TEXT,
                                embedding vector(384))"""
        )

        # Store all the generated embeddings back into the database.
        for index, row in product_embeddings.iterrows():
            await conn.execute(
                "INSERT INTO clothing_embeddings (index, clothing_id, content, embedding) VALUES ($1, $2, $3, $4)",
                row["index"],
                row["clothing_id"],
                row["content"],
                np.array(row["embedding"]),
            )

        await conn.close()


# Run the SQL commands now.
await main()  # type: ignore

/usr/local/lib/python3.11/dist-packages/google/cloud/sql/connector/instance.py:330: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  expiration = x509.not_valid_after


In [ ]:
# Both the HNSW index and the IVFFLAT are for ANN (approximate nearest neighbor)
# @markdown Create an HNSW index on the `product_embeddings` table:
m =  24 # @param {type:"integer"}
ef_construction = 100  # @param {type:"integer"}
operator =  "vector_cosine_ops"  # @param ["vector_cosine_ops", "vector_l2_ops", "vector_ip_ops"]

# Quick input validations.
assert m, "⚠️ Please input a valid value for m."
assert ef_construction, "⚠️ Please input a valid value for ef_construction."
assert operator, "⚠️ Please input a valid value for operator."

from pgvector.asyncpg import register_vector
import asyncio
import asyncpg
from google.cloud.sql.connector import Connector


async def main():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await register_vector(conn)

        # Create an HNSW index on the `product_embeddings` table.
        await conn.execute(
            f"""CREATE INDEX ON clothing_embeddings
              USING hnsw(embedding {operator})
              WITH (m = {m}, ef_construction = {ef_construction})
            """
        )

        await conn.close()


# Run the SQL commands now.
await main()  # type: ignore

In [ ]:
# @markdown Create an IVFFLAT index on the `product_embeddings` table:
lists =  100 # @param {type:"integer"}
operator =  "vector_cosine_ops"  # @param ["vector_cosine_ops", "vector_l2_ops", "vector_ip_ops"]

# Quick input validations.
assert lists, "⚠️ Please input a valid value for lists."

from pgvector.asyncpg import register_vector
import asyncio
import asyncpg
from google.cloud.sql.connector import Connector


async def main():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await register_vector(conn)

        # Create an IVFFLAT index on the `product_embeddings` table.
        await conn.execute(
            f"""CREATE INDEX ON clothing_embeddings
              USING ivfflat(embedding {operator})
              WITH (lists = {lists})
            """
        )

        await conn.close()


# Run the SQL commands now.
await main()  # type: ignore

# Test RAG

In [ ]:
import torch
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {
    "device": "cuda" if torch.cuda.is_available() else "cpu",
   # "show_progress": True  # Enable progress bar
}
if torch.cuda.is_available():
  print('encoding with cuda')
else:
  print('encoding with cpu')
embeddings_service = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs)


encoding with cpu
encoding with cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# @markdown Enter a short description of the clothing to search for within a specified price range:
search = "Absolutely wonderful - silky and sexy and comfortable"  # @param {type:"string"}
min_age = 0  # @param {type:"integer"}
max_age = 100  # @param {type:"integer"}

# input validations.
assert search, "⚠️ Please input a valid input search text"
from langchain_google_vertexai import VertexAIEmbeddings
# from langchain.embeddings import VertexAIEmbeddings
from google.cloud import aiplatform

aiplatform.init(project=f"{project_id}", location=f"{region}")

# embeddings_service = VertexAIEmbeddings(model="text-embedding-004")
qe = embeddings_service.embed_query(search)
from pgvector.asyncpg import register_vector
import asyncio
import asyncpg
from google.cloud.sql.connector import Connector

matches = []

async def main():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await register_vector(conn)
        similarity_threshold = 0.5
        num_matches = 50

        # Find similar products to the query using cosine similarity search
        # over all vector embeddings. This new feature is provided by `pgvector`.
        results = await conn.fetch(
            """
                            WITH vector_matches AS (
                              SELECT index, 1 - (embedding <=> $1) AS similarity
                              FROM clothing_embeddings
                              WHERE 1 - (embedding <=> $1) > $2
                              ORDER BY similarity DESC
                              FETCH FIRST $3 ROWS ONLY
                            )
                            SELECT clothing_id, review, age FROM clothing
                            WHERE index IN (SELECT index FROM vector_matches)
                            AND age >= $4 AND age <= $5
                            """,
            qe,
            similarity_threshold,
            num_matches,
            min_age,
            max_age,
        )

        if len(results) == 0:
            raise Exception("Did not find any results. Adjust the query parameters.")
        for r in results:
            matches.append(
                {
                    "clothing_id": r["clothing_id"],
                    "review": r["review"],
                    #"list_price": round(r["list_price"], 2),
                }
            )

        await conn.close()


# Run the SQL commands now.
await main()  # type: ignore


# Show the results for similar products that matched the user query.
matches = pd.DataFrame(matches)
matches.head(5)

/usr/local/lib/python3.11/dist-packages/google/cloud/sql/connector/instance.py:330: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  expiration = x509.not_valid_after


NameError: name 'pd' is not defined

# LLM and RAG

In [ ]:
# @markdown Enter the user search query in a simple English text. The age filter filters review from a age group
# Please fill in these values.
user_query = "a dress for summer that is comfortable and classic"  # @param {type:"string"}
min_age = 0  # @param {type:"integer"}
max_age = 100  # @param {type:"integer"}

# Quick input validations.
assert user_query, "⚠️ Please input a valid input search text"

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import torch
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {
    "device": "cuda" if torch.cuda.is_available() else "cpu",
   # "show_progress": True  # Enable progress bar
}
if torch.cuda.is_available():
  print('encoding with cuda')
else:
  print('encoding with cpu')
embeddings_service = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs)

qe = embeddings_service.embed_query(user_query)

encoding with cpu


In [ ]:
from pgvector.asyncpg import register_vector
import asyncio
import asyncpg
from google.cloud.sql.connector import Connector

matches = []

async def main():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await register_vector(conn)
        similarity_threshold = 0.5 # was 0.7 but too high
        num_matches = 3

        # Find similar products to the query using cosine similarity search
        # over all vector embeddings, then use index as handle to associate two tables
        results = await conn.fetch(
            """
                            WITH vector_matches AS (
                              SELECT index, 1 - (embedding <=> $1) AS similarity
                              FROM clothing_embeddings
                              WHERE 1 - (embedding <=> $1) > $2
                              ORDER BY similarity DESC
                              FETCH FIRST $3 ROWS ONLY
                            )
                            SELECT clothing_id, review, age FROM clothing
                            WHERE index IN (SELECT index FROM vector_matches)
                            AND age >= $4 AND age <= $5
                            """,
            qe,
            similarity_threshold,
            num_matches,
            min_age,
            max_age,
        )

        if len(results) == 0:
            raise Exception("Did not find any results. Adjust the query parameters.")

        for r in results:
            # Collect the description for all the matched similar items
            matches.append(
                f"""The clothing ID is {r["clothing_id"]}.
                          The review is ${r["review"]}.
                         """
            )
        await conn.close()


# Run the SQL commands now.
await main()  # type: ignore

# Show the results for similar products that matched the user query.
matches

/usr/local/lib/python3.11/dist-packages/google/cloud/sql/connector/instance.py:330: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  expiration = x509.not_valid_after


['The clothing ID is 1086.\n                          The review is $The summer dress-I saw this online and thought the idea was super cute, but was not sure if it would be worth the money because it looks typical. but, when i went to the store to try it on, i saw it was anything but typical. the fit is perfect and the fabric so soft. i wanted the dress in the green, but they were already sold out in my size at the store, so i had to order it online. i cannot wait until it comes in! i will wear this often during the summer..\n                         ',
 'The clothing ID is 1103.\n                          The review is $This is a fantastic summer wear all the time dress-I love this dress. you can wear it for everyday or you can put some bling with it and dress it up. it fits very well. flattering. i highly recommend!!.\n                         ',
 "The clothing ID is 1078.\n                          The review is $Summer dress-I bought this dress to wear as a casual summer staple. i 

In [ ]:
# Using LangChain for summarization and efficient context building.
from langchain_google_vertexai import ChatVertexAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain import PromptTemplate, LLMChain
from IPython.display import display, Markdown

llm = ChatVertexAI(
    model="gemini-1.0-pro",
    #temperature=0,
    project=project_id)

map_prompt_template = """
              You will be given a piece of review text of a clothing product.
              This description is enclosed in triple backticks (```).
              Using this description only, extract the ID and the features of the clothing.

              ```{text}```
              SUMMARY:
              """
map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                You will be given about 3 descriptions of different clothings
                enclosed in triple backticks (```) and a question enclosed in
                double backticks(``).
                For every clothing item that you received,
                answer how the clothing match the question based its features.
                Mention all clothing items that you received.
                The answer should be in as much detail as possible.
                You should only use the information in the description.
                Your answer should include the ID of the clothings and their features.
                Your answer should be less than 300 words.
                Your answer should be in Markdown in a numbered list format.
                Do not include a conclusion or a note.


                Description:
                ```{text}```


                Question:
                ``{user_query}``


                Answer:
                """
combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text", "user_query"]
)

docs = [Document(page_content=t) for t in matches]
chain = load_summarize_chain(
    llm, chain_type="map_reduce", map_prompt=map_prompt, combine_prompt=combine_prompt
)
answer = chain.run(
    {
        "input_documents": docs,
        "user_query": user_query,
    }
)


display(Markdown(answer))

## Dresses for a Comfortable and Classic Summer

Based on the descriptions you provided, here are the dresses that match your request and their relevant features:

1. **ID: 1086** 
    * This dress is perfect for summer with its soft fabric and comfortable fit.
    * The reviewer's experience suggests that it is flattering and unique, despite initial impressions online. 
    * While the specific color mentioned is green, other colors likely exist.

2. **ID: 1103** 
    * This dress is explicitly described as fantastic summer wear.
    * It is versatile, allowing for both casual and dressed-up occasions.
    * The reviewer highlights its flattering fit and highly recommends it.

3. **ID: 1078** 
    * While not explicitly mentioned as a summer dress, its vivid colors and classic style with a twist suggest suitability for the season.
    * The reviewer is happy with the quality and price, finding it a good value.
    * It is important to note the slightly tight fit in the bust, which may not be universally comfortable. 


In [ ]:
# @markdown Clean-up and delete the Cloud SQL instance.
!gcloud sql instances patch {instance_name} --no-deletion-protection
!gcloud sql instances delete {instance_name} --quiet

The following message will be used for the patch API method.
{"name": "pg15-pgvector-demo", "project": "chatbot-with-rag-447503", "settings": {"deletionProtectionEnabled": false}}
Updated [https://sqladmin.googleapis.com/sql/v1beta4/projects/chatbot-with-rag-447503/instances/pg15-pgvector-demo].
Deleted [https://sqladmin.googleapis.com/sql/v1beta4/projects/chatbot-with-rag-447503/instances/pg15-pgvector-demo].
